<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/Dataset_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from os import listdir
from scipy.io import loadmat
filename = []
for mat_file in listdir('/content/drive/MyDrive/sound2meg/mat_files'):
  filename.append(mat_file)
sizes = []
for file in filename:
  mat_file = loadmat('/content/drive/MyDrive/sound2meg/mat_files/' + file)  #Was really slow to import every file to check the size, hence not included in the class
  Signal = mat_file['data']
  sizes.append(Signal.shape[2])

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.io import loadmat
from os import listdir
import numpy as np
from sklearn.preprocessing import RobustScaler

def Cumulative(lists):
    cu_list = []
    length = len(lists)
    cu_list = [sum(lists[0:x:1]) for x in range(0, length+1)]
    return cu_list[1:]

class Sound2MEGDataset(Dataset):
  def __init__(self, meg_files_path, sizes):
    #self.wav_files = wav_files
    self.meg_files_path = meg_files_path
    self.sizes = sizes
  def __len__(self):
    return sum(self.sizes)
  def __getitem__(self, idx):
    filename = []
    for mat_file in listdir('/content/drive/MyDrive/sound2meg/mat_files'):
      filename.append(mat_file)
    for i in range(len(self.sizes)):
      if Cumulative(self.sizes)[i] >= idx:
        idx_file = i
        break
    mat_file = loadmat(self.meg_files_path + filename[idx_file])
    idx_sound = idx - Cumulative(self.sizes)[idx_file - 1]
    mat_file = mat_file['data']
    MEG_Signal = np.float32(mat_file[:, :, idx_sound])
    transformer = RobustScaler().fit(MEG_Signal)
    MEG_Signal = transformer.transform(MEG_Signal)
    return torch.from_numpy(MEG_Signal)

Dataset = Sound2MEGDataset('/content/drive/MyDrive/sound2meg/mat_files/', sizes)  #Can make a vector of sizes in the directory and import it from there
Sample = Dataset[1245]
print(Sample.shape)


7
torch.Size([273, 360])


In [ ]:
print(sizes)

[147, 168, 171, 166, 167, 178, 170, 152, 180, 99, 165, 169, 170, 169, 170, 171, 164, 174, 160, 174, 183, 171, 178, 162, 155, 167, 163, 154, 164, 90, 174, 168, 162, 173, 163, 170, 170, 169, 170, 178, 174, 170, 163, 162, 164, 170, 179, 164, 178, 146, 22, 177, 170, 160, 169, 178, 153, 166, 172, 158, 167, 164, 105, 0, 174, 169, 169, 166, 49, 172, 168, 164, 166, 166, 165, 177, 161, 166, 169, 168, 178, 165, 168, 160, 171, 171, 169, 165, 173, 157, 170, 171, 157, 171, 171, 172, 156, 175, 158, 168, 168, 164]
